In [1]:
## Импорт необходимых библиотек 

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import re

from datetime import datetime

In [2]:
## Подгрузка черновой базы статьи 

data = pd.read_csv('105_2013_2019_features_v3_2.csv')

C:\Users\hamri\AppData\Local\Temp\ipykernel_3004\1907425286.py:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('105_2013_2019_features_v3_2.csv')


In [3]:
## Подгрузка собранных идентификаторов релевантных уголовных дел
## fast_id -- идентификаторы дел, размеченных в финальной базе данных 
## model_id -- идентификаторы дел, размеченных моделью как релевантные

fast_id = pd.read_csv('fast_women_cases_id_2018_diplom.csv')
model_id = pd.read_csv('women_cases_id_2018_diplom.csv')

In [4]:
## Подгрузка финальной базы данных статьи  

data_reg = pd.read_csv('regression_data.csv')

In [6]:
## Отбор фамилий и инициалов подсудимых в релевантных уголовных делах

surnames = data[
    data['ID'].isin(fast_id['ID'].tolist() + model_id['ID'].tolist())
]['ФИО'].tolist()

In [16]:
## Отбор года вынесения приговора в релевантных уголовных делах

years = data[
    data['ID'].isin(fast_id['ID'].tolist() + model_id['ID'].tolist())
]['year'].tolist()

In [7]:
## Подгрузка необходимых для веб-скрейпинга библиотек

from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from getpass import getpass
from bs4 import BeautifulSoup
from time import sleep
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from tqdm import tqdm 
import requests

In [1274]:
## Сбор ссылок по запросам в браузере 
## Формирование слов для внесения в запрос
rel_phrases = ['убийство', 'смерть', 'домашнее насилие', 'суд', 'наказание', 'преступление']
posts_all = pd.DataFrame()

## Проход по всем фамилиям и годам заключения
for surname, year in tqdm(zip(surnames, years)):
    ## Проход по всем релевантным словам
    for phrase in rel_phrases:
        ## Загрузка страницы 
        br = wd.Chrome(service=Service(ChromeDriverManager().install()))
        ## Инициализация запроса
        br.get('https://www.google.com/search?q=%D0%B2%D0%BA%D0%BE%D0%BD%D1%82%D0%B0%D0%BA%D1%82%D0%B5&oq=%D0%B2%D0%BA%D0%BE%D0%BD%D1%82%D0%B0%D0%BA&gs_lcrp=EgZjaHJvbWUqFQgAEAAYQxiDARjjAhixAxiABBiKBTIVCAAQABhDGIMBGOMCGLEDGIAEGIoFMhgIARAuGEMYgwEYxwEYsQMY0QMYgAQYigUyBggCEEUYOTINCAMQABiDARixAxiABDIHCAQQABiABDINCAUQABiDARixAxiABDIGCAYQBRhAMgYIBxBFGD2oAgCwAgA&sourceid=chrome&ie=UTF-8')
        ## Имитация заполнения строки поиска и нажатия кнопки
        search_box = br.find_element(By.NAME, "q")
        search_box.send_keys(f" {phrase} {surname} {year} год")
        search_box.submit()
        ## Имитация ожидания 
        br.implicitly_wait(2)
        
        ## Загрузка кода страницы
        html = br.page_source
        soup = BeautifulSoup(html)
        
        ## Сбор ссылок со страницы
        links = []
        for link_info in soup.find_all('a'):
            links.append(link_info.get('href'))
        posts = pd.DataFrame(links)
        posts['surname'] = surname
        
        posts_all = pd.concat([posts_all, posts])
        ## Имитация ожидания 
        br.implicitly_wait(2)

In [22]:
## Удаление дубликатов

posts_all = posts_all.dropna().drop_duplicates()

In [23]:
## Нахождение релевантных подстрок для определения ссылок на новостные публикации в ВКонтакте 

posts_all['is_vk'] = posts_all[0].apply(lambda x: 1 if 'vk.com' in x else 0)
posts_all['is_wall'] = posts_all[0].apply(lambda x: 1 if 'wall' in x else 0)
posts_all['is_vk_wall'] = posts_all[0].apply(lambda x: 1 if 'vk.com/wall' in x else 0)

In [51]:
## Формирование токена для доступа к ВК API

app_id = ## Ваше название приложения
url = f"https://oauth.vk.com/authorize?client_id={app_id}&display=page&redirect_uri=http://oauth.vk.com/blank.html&scope=all&response_type=token"
print(url)

https://oauth.vk.com/authorize?client_id=51542144&display=page&redirect_uri=http://oauth.vk.com/blank.html&scope=all&response_type=token


In [52]:
## Определение токена

full_link = ''
token = full_link.split("access_token=")[1].split("&")[0]

In [26]:
## Функция для выгрузки информации о релевантных новостных публикациях в ВКонтакте

def get_charac(x):
    x = '-' + x.split('-')[1]
    main_news = f"https://api.vk.com/method/wall.getById"
    params = {
        'access_token': token,
        'v': '5.199',
        'posts': x,
    }

    req_wall = requests.get(main_news, params = params)
    data = req_wall.json()
    try:
        data = pd.DataFrame(data['response']['items'])
    except KeyError:
        return pd.DataFrame([0])
    
    return data

In [40]:
## Функция приведения ссылок к одному формату для передачи в API 

def get_post_info(x):
    x = x.split('?')[0].split('-')[-1]
    if x.replace('_', '').isdigit():
        return x
    else:
        return 0

In [41]:
## Отбор релевантных ссылок 

posts_vk = posts_all[
    posts_all['is_vk_wall'] == 1
].copy()

In [42]:
## Форматирование ссылок с помощью функции 

posts_vk['post_link'] = posts_vk[0].apply(
    lambda x: get_post_info(x)
)
posts_vk['clear_link'] = posts_vk[0].apply(lambda x: x.split('?')[0]).tolist()

In [65]:
## Отбор релевантных ссылок 

posts_link = posts_vk[
    posts_vk['post_link'] != 0
]['clear_link'].tolist()

In [71]:
## Сбор информации о публикациях 

posts_list = []

for link in tqdm(posts_link):
    link_info = get_charac(link)
    link_info['link'] = link
    posts_list.append(link_info)

100%|████████████████████████████████████████████████████████████████████████████████| 733/733 [01:45<00:00,  6.93it/s]


In [74]:
## Формирование единого датафрейма информации о собранных постах

etalon_cols = posts_list[0].columns[:-1]
posts_final_list = []
for post_info in tqdm(posts_list):
    if len(post_info.columns) < 10:
        for col in etalon_cols:
            post_info[col] = '-'
    posts_final_list.append(post_info)
posts_charac = pd.concat([i for i in posts_final_list])

100%|███████████████████████████████████████████████████████████████████████████████| 733/733 [00:03<00:00, 207.53it/s]


In [85]:
## Собранные из постов в ВКонтакте характеристики  

posts_charac.columns

Index([       'inner_type',             'donut',          'comments',
           'marked_as_ads',   'short_text_rate',              'hash',
         'has_translation',              'type',       'attachments',
                    'date',           'from_id',                'id',
             'is_favorite',             'likes',   'reaction_set_id',
               'reactions',          'owner_id',       'post_source',
               'post_type',           'reposts',              'text',
                   'views',              'link',   'carousel_offset',
       'donut_miniapp_url',         'signer_id',            'edited',
                         0,         'copyright',        'is_deleted',
          'deleted_reason',   'deleted_details',      'copy_history',
               'zoom_text'],
      dtype='object')

In [464]:
import numpy as np

In [88]:
## Объединение изначального датасета по ссылкам с датасетом информации о постах 
## Используется inner join для отбрасывания 

posts_vk = posts_vk.rename(columns = {0: 'link'})

final_posts_vk = posts_vk.merge(
    posts_charac,
    on='link'
).copy()

In [441]:
## Объединение полученных данных с датами вынесения приговора и поступления уголовного дела 
## для фильтрации нерелевантных наблюдений

data['surname'] = data['ФИО']
final_posts_vk_no_dup_info = final_posts_vk.merge(data[[
    'surname', 'Дата поступления', 'Дата решения'
]], on='surname').copy().dropna(subset=['Дата поступления', 'Дата решения'])

In [442]:
## Функция перевода даты в формат epoch 

import calendar, time
def get_epoch_time(x):
   return calendar.timegm(time.strptime(x, '%d.%m.%Y'))

In [454]:
## Применение функции перевода даты в формат epoch к дате поступления уголовного дела и дате вынесения приговора 

final_posts_vk_no_dup_info['start_date'] = final_posts_vk_no_dup_info['Дата поступления'].apply(
    lambda x: get_epoch_time(x)
)
final_posts_vk_no_dup_info['end_date'] = final_posts_vk_no_dup_info['Дата решения'].apply(
    lambda x: get_epoch_time(x)
)

## Проверка на наличие фамилии в тексте поста
final_posts_vk_no_dup_info['is_surname'] = final_posts_vk_no_dup_info.apply(
    lambda x: 1 if x['surname'][:-2].split()[0] in x['text'] else 0,
    axis=1
)
## Определение убийства в посте 
final_posts_vk_no_dup_info['is_crime'] = final_posts_vk_no_dup_info['text'].apply(
    lambda x: 1 if 'уби' in x or 'смерт' in x or 'насили' in x else 0 
)

## Проверка на срок выкладывания публикации 
final_posts_vk_no_dup_info['is_actual'] = final_posts_vk_no_dup_info.apply(
    lambda x: 1 if x['start_date'] <= x['date'] and x['date'] <= x['end_date'] else 0,
    axis=1
)

In [460]:
## Отбор релевантных ссылок с учетом
## Временных ограничений, фамилии и тематики поста

rel_links = final_posts_vk_no_dup_info[
    (final_posts_vk_no_dup_info['is_actual'] == 1) &\
    (final_posts_vk_no_dup_info['is_surname'] == 1 ) &\
    (final_posts_vk_no_dup_info['is_crime'] == 1)
].copy()['link']

In [462]:
## Отбор характеристик релевантных ссылок

final_posts_vk_no_dup = final_posts_vk_no_dup_info[
    final_posts_vk_no_dup_info['link'].isin(rel_links)
]

In [133]:
posts_vk_rel_cols = final_posts_vk_no_dup[[
    'surname', 
    'comments',
    'likes',
    'views',
    'reposts'
]].copy()

In [134]:
posts_vk_rel_cols['comments_cnt'] = posts_vk_rel_cols['comments'].apply(
    lambda x: x['count'] if isinstance(x, dict) else 0
)
posts_vk_rel_cols['likes_cnt'] = posts_vk_rel_cols['likes'].apply(
    lambda x: x['count'] if isinstance(x, dict) else 0
)
posts_vk_rel_cols['views_cnt'] = posts_vk_rel_cols['views'].apply(
    lambda x: x['count'] if isinstance(x, dict) else 0
)
posts_vk_rel_cols['reposts_cnt'] = posts_vk_rel_cols['reposts'].apply(
    lambda x: x['count'] if isinstance(x, dict) else 0
)

In [135]:
## Группировка статистик релевантных постов по фамилии обвиняемого 

vk_stats = pd.DataFrame(posts_vk_rel_cols.groupby('surname').agg(
    {
        'views_cnt': ['sum'],
        'comments_cnt': ['sum'],
        'likes_cnt': ['sum'],
        'reposts_cnt': ['sum']
    }
).values, columns = [
    'views_cnt', 'comments_cnt', 'likes_cnt', 'reposts_cnt'
])

## Определение фамилии обвиняемого 
vk_stats['ФИО'] = posts_vk_rel_cols.groupby('surname').agg(
    {
        'views_cnt': ['sum'],
        'comments_cnt': ['sum'],
        'likes_cnt': ['sum'],
        'reposts_cnt': ['sum']
    }
).index.tolist()

In [190]:
## Присоединение к изначальной базе аггрегированных статистик релевантных постов
## Повторный отбор наблюдений для проверки
## Чистка от случаев, где невозможно определить целевую переменную -- срок заключения

dataset = data.merge(vk_stats,on='ФИО').drop_duplicates(subset=['ФИО']).copy()
dataset = dataset[
    dataset['year'].isin([2017, 2018, 2019])
].dropna(subset='Дата поступления').copy()
dataset = dataset[
    (~dataset['sentence_period_months_int'].isin([
        'ЧИСЛО МЕСЯЦЕВ В СРОКЕ НЕ УСТАНОВЛЕНО', 'ДАННЫЕ ИЗЪЯТЫ'
    ])) &
    (~dataset['sentence_period_years_int'].isin([
        'ЧИСЛО МЕСЯЦЕВ В СРОКЕ НЕ УСТАНОВЛЕНО', 'ДАННЫЕ ИЗЪЯТЫ'
    ])) 
].copy()

In [ ]:
## Датасет на базе черновой базе данных статьи

In [198]:
## Отбор релевантных колонок из промежуточной базы данных 

dataset_rel = dataset[[
    'ID', 'Дата поступления', 'Дата решения', 'text', 'ФИО', 'gender_judge',
    'soft', 'hard', 'recidivism', 'guilty_plea', 'alcohol', 'year', 
    'sentence_period_months_int', 'sentence_period_years_int', 'views_cnt',
    'comments_cnt', 'likes_cnt', 'reposts_cnt',
]].copy()

In [200]:
dataset_final = dataset_rel.copy()

In [ ]:
## Кодирование переменных

In [213]:
## Определение количества месяцев и лет срока заключения
## Формирование срока заключения и фильтрация значений 0 

dataset_final['months_final'] = dataset_final['sentence_period_months_int'].apply(
    lambda x: int(x.split('.')[0]) if x.replace('.', '').isdigit() else 0
)

dataset_final['years_final'] = dataset_final['sentence_period_years_int'].apply(
    lambda x: int(x.split('.')[0]) if x.replace('.', '').isdigit() else 0
)

dataset_final['length_of_imprisonment'] = dataset_final['years_final'] * 12 + dataset_final['months_final']
dataset_final = dataset_final[
    dataset_final['length_of_imprisonment'] != 0
]

In [373]:
## Перекодирование используемых в анализе переменных

dataset_final['gender_judge_bin'] = dataset_final['gender_judge'].apply(
    lambda x: 1 if x == 'Мужской' else 0
)
dataset_final['soft_bin'] = dataset_final['soft'].apply(
    lambda x: 1 if x == 'Смягчающие признаны' else 0 
)
dataset_final['hard_bin'] = dataset_final['hard'].apply(
    lambda x: 1 if x == 'Отягчающие признаны' else 0 
)
dataset_final['recidivism_bin'] = dataset_final['recidivism'].apply(
    lambda x: 1 if x == 'есть судимость' else 0 
)
dataset_final['alcohol_bin'] = dataset_final['alcohol'].apply(
    lambda x: 1 if x == 'Алкоголь присутствует' else 0 
)
dataset_final['guilty_plea_order'] = dataset_final['guilty_plea'].apply(
    lambda x: 2 if x == 'Вина признана полностью' else 1 if x == 'Вина признана частично' else 0 
)

C:\Users\hamri\AppData\Local\Temp\ipykernel_3004\2283349991.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_final['soft_bin'] = dataset_final['soft'].apply(
C:\Users\hamri\AppData\Local\Temp\ipykernel_3004\2283349991.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_final['hard_bin'] = dataset_final['hard'].apply(
C:\Users\hamri\AppData\Local\Temp\ipykernel_3004\2283349991.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

In [551]:
## Преобразования для правильного объединения таблиц 
## Удаление дубликатов

dataset_final = dataset_final.merge(
    data[['ID', 'ФИО', 'Дата поступления']], 
    on=['ID', 'Дата поступления']
).drop_duplicates().copy()

dataset_final = dataset_final.merge(data[[
    'ФИО', 'recidivism_type'
]], on='ФИО', how='left').copy()

In [559]:
## Функция кодирования степени рецидивизма и применение к колонке

def code_recidivism(x):
    if x == 'нет судимости': 
        return 0
    elif x == 'судимость не признана рецидивом': 
        return 1
    elif x == 'рецидив': 
        return 2
    elif x == 'опасный рецидив':
        return 3
    else:
        return 4
    
dataset_final['recidivism_type'] = dataset_final['recidivism_type'].apply(
    lambda x: code_recidivism(x)
)

In [561]:
## Присоединение переменных оружия к полученной базе 

dataset_final = dataset_final.merge(data[['ФИО', 'weapon_knife', 'weapon_gun']], on='ФИО').copy()
dataset_final = dataset_final.merge(data[['ФИО', 'weapon_hands', 'weapon_legs']], on='ФИО').copy()
dataset_final = dataset_final.merge(
    data[['ФИО', 'weapon_other', 'weapon_not_used']], on='ФИО'
).drop_duplicates(subset='ID').copy()
dataset_final = dataset_final.drop_duplicates(subset='ID').copy()

In [612]:
## Добавление количества постов на уголовное дело 

dataset_final['surname'] = dataset_final['ФИО']
dataset_final_new = dataset_final.merge(
    posts_vk_rel_cols['surname'].value_counts().reset_index(),
    on='surname'
).copy()
dataset_final_new = dataset_final_new.rename(columns={'count': 'posts_count'})

In [723]:
## Формирование списка характеристик медиа
## Смещение значения переменных в ноль

media_cols = ['views_cnt', 'likes_cnt', 'comments_cnt', 'reposts_cnt', 'posts_count']
for i in media_cols:
    dataset_final_new[i] = dataset_final_new[i] - min(dataset_final_new[i]) + 0.00001

In [787]:
## Отбор релевантных колонок

data_no_pub = data[
     (data['ID'].isin(fast_id['ID'].tolist() + model_id['ID'].tolist())) &\
    (~data['ID'].isin(dataset_final_new['ID']))
][['ID', 'ФИО', 'surname', 'Дата решения', 'Дата поступления', 'text', 'gender_judge', 'gender_victim', 'soft',
       'hard', 'recidivism', 'guilty_plea', 'alcohol', 'year',
       'sentence_period_months_int', 'sentence_period_years_int', 'recidivism_type',
 'weapon_knife', 'weapon_gun',  'weapon_hands', 'weapon_legs', 'weapon_other', 'weapon_not_used' ]].copy()

In [789]:
## Удаление наблюдений, у которых нельзя определить срок заключения

data_no_pub_new = data_no_pub[
    (~data_no_pub['sentence_period_months_int'].isin([
        'ЧИСЛО МЕСЯЦЕВ В СРОКЕ НЕ УСТАНОВЛЕНО', 'ДАННЫЕ ИЗЪЯТЫ'
    ])) &
    (~data_no_pub['sentence_period_years_int'].isin([
        'ЧИСЛО МЕСЯЦЕВ В СРОКЕ НЕ УСТАНОВЛЕНО', 'ДАННЫЕ ИЗЪЯТЫ'
    ])) 
].copy()

In [791]:
## Формирование целевой переменной 

data_no_pub_new['months_final'] = data_no_pub_new['sentence_period_months_int'].apply(
    lambda x: int(x.split('.')[0]) if x.replace('.', '').isdigit() else 0
)
data_no_pub_new['years_final'] = data_no_pub_new['sentence_period_years_int'].apply(
    lambda x: int(x.split('.')[0]) if x.replace('.', '').isdigit() else 0
)
data_no_pub_new['length_of_imprisonment'] = data_no_pub_new['years_final'] * 12 + data_no_pub_new['months_final']
data_no_pub_new = data_no_pub_new[
    data_no_pub_new['length_of_imprisonment'] != 0
]

In [795]:
## Кодирование переменных 

data_no_pub_new['gender_judge_bin'] = data_no_pub_new['gender_judge'].apply(
    lambda x: 1 if x == 'Мужской' else 0
)
data_no_pub_new['soft_bin'] = data_no_pub_new['soft'].apply(
    lambda x: 1 if x == 'Смягчающие признаны' else 0 
)
data_no_pub_new['hard_bin'] = data_no_pub_new['hard'].apply(
    lambda x: 1 if x == 'Отягчающие признаны' else 0 
)
data_no_pub_new['recidivism_bin'] = data_no_pub_new['recidivism'].apply(
    lambda x: 1 if x == 'есть судимость' else 0 
)
data_no_pub_new['alcohol_bin'] = data_no_pub_new['alcohol'].apply(
    lambda x: 1 if x == 'Алкоголь присутствует' else 0 
)
data_no_pub_new['guilty_plea_order'] = data_no_pub_new['guilty_plea'].apply(
    lambda x: 2 if x == 'Вина признана полностью' else 1 if x == 'Вина признана частично' else 0 
)
data_no_pub_new['recidivism_type'] = data_no_pub_new['recidivism_type'].apply(
    lambda x: code_recidivism(x)
)

In [843]:
## Объединение двух частей датасета в один
## На данном этапе учитываются как дела с наличием упоминаний в ВК, так и без 

data_media = pd.concat([
    dataset_final_new,
    data_no_pub_new
]).copy()
data_media = data_media.dropna(subset='Дата решения')
data_media = data_media.fillna(0)

In [873]:
## Повторная проверка возможности составить целевую переменную 

data_media = data_media[
    (~data_media['sentence_period_months_int'].isin(['ЧИСЛО ЛЕТ В СРОКЕ НЕ УСТАНОВЛЕНО']))
].copy()
data_media = data_media[
    (~data_media['sentence_period_years_int'].isin(['ЧИСЛО ЛЕТ В СРОКЕ НЕ УСТАНОВЛЕНО']))
].copy()

In [932]:
## Бинаризация переменных степени признания вины и степени рецидива 

data_media_new = pd.concat([data_media, pd.get_dummies(data_media['guilty_plea'])], axis=1).copy()
data_media_new =  pd.concat([data_media_new, pd.get_dummies(data_media['recidivism_type'])], axis=1).copy()

In [934]:
## Форматирование названия колонок 

data_media_new = data_media_new.rename(
    columns = {
        'Вина не признана': 'guilty_plea_0',
        'Вина признана частично': 'guilty_plea_1',
        'Вина признана полностью': 'guilty_plea_2',
        0: 'recidivism_type_0',
        1: 'recidivism_type_1',
        2: 'recidivism_type_2',
        3: 'recidivism_type_3',
        4: 'recidivism_type_4',
    }
)
bin_list = ['guilty_plea_0',
'guilty_plea_1',
'guilty_plea_2',
'recidivism_type_0',
'recidivism_type_1',
'recidivism_type_2',
'recidivism_type_3',
'recidivism_type_4',]

for col in bin_list:
    data_media_new[col] = data_media_new[col].apply(lambda x: int(x))

In [935]:
## Смещение целевой переменной в ноль для интерпретации

data_media_new['length_of_imprisonment_null'] = data_media_new['length_of_imprisonment'] - min(data_media_new['length_of_imprisonment'])

## Присваивание нужного типа данных дихотомическим переменным
bin_list = ['guilty_plea_0',
'guilty_plea_1',
'guilty_plea_2',
'recidivism_type_0',
'recidivism_type_1',
'recidivism_type_2',
'recidivism_type_3',
'recidivism_type_4',]
for col in bin_list:
    data_media_new[col] = data_media_new[col].apply(lambda x: int(x))

In [1029]:
data_media_new.to_csv('data_media_final.csv')

In [ ]:
## Датасет на базе финального в статье 

In [1045]:
## Отбор в финальной базе данных статьи с помощью идентификаторов наблюдений, 
## которые релевантны в качестве выборки 

data_reg_media = data_reg[
    data_reg['ID'].isin(
        data_media['ID'].tolist()
    )
].copy()

In [1098]:
## Кодирование переменных 

data_reg_media['recidivism_type_order'] = data_reg_media['recidivism_type'].apply(lambda x: x.split()[0])
data_reg_media['guilty_plea_order'] = data_reg_media['guilty_plea'].apply(lambda x: x.split()[0])
data_reg_media['soft_bin'] = data_reg_media['soft_binary'].apply(lambda x: int(x.split()[0]))
data_reg_media['hard_bin'] = data_reg_media['hard_binary'].apply(lambda x: int(x.split()[0]))
data_reg_media['child_bin'] = data_reg_media['children'].apply(lambda x: x.split()[0])
data_reg_media['child_bin'] = data_reg_media['child_bin'].apply(lambda x: int(x))

In [1076]:
## Исправление колонок после дихотомизации переменной степени признания вины

data_reg_media = pd.concat([data_reg_media, pd.get_dummies(data_reg_media['guilty_plea_order'])], axis=1).copy()
data_reg_media = data_reg_media.rename(
    columns = {
        '0': 'guilty_plea_0',
        '1': 'guilty_plea_1',
        '2': 'guilty_plea_2',
    }
)

In [1081]:
## Исправление колонок после дихотомизации переменной степени рецидива

data_reg_media =  pd.concat([data_reg_media, pd.get_dummies(data_reg_media['recidivism_type_order'])], axis=1).copy()
data_reg_media = data_reg_media.rename(
    columns = {
        '0': 'recidivism_type_0',
        '1': 'recidivism_type_1',
        '2': 'recidivism_type_2',
        '3': 'recidivism_type_3',
        '4': 'recidivism_type_4',
    }
)

In [1083]:
## Присвоение нужного типа данных дихотомическим переменным 

bin_list = ['guilty_plea_0',
'guilty_plea_1',
'guilty_plea_2',
'recidivism_type_0',
'recidivism_type_1',
'recidivism_type_2',
'recidivism_type_3',
'recidivism_type_4',]
for col in bin_list:
    data_reg_media[col] = data_reg_media[col].apply(lambda x: int(x))

In [1086]:
## Кодирование переменных 
## Смещение целевой переменной в ноль 

data_reg_media['gender_judge_bin'] = data_reg_media['gender_judge'].apply(
    lambda x: 1 if x == 'Мужской' else 0
)
data_reg_media['posts_bin'] = data_reg_media['posts_count'].apply(lambda x: 1 if x > 0 else 0)
data_reg_media['sentence_period_whole_null'] = data_reg_media['sentence_period_whole'] - data_reg_media['sentence_period_whole'].min()

In [1176]:
data_reg_media.to_csv('final_data_diplom.csv')

In [1277]:
data_reg_media.columns

Index(['Unnamed: 0', 'ID', 'gender_victim', 'gender_judge', 'gender_offender',
       'recidivism_type', 'guilty_plea', 'soft_binary', 'hard_binary',
       'rel_type', 'year', 'children', 'sentence_period_whole', 'region',
       'Наименование суда', 'likes_cnt', 'views_cnt', 'reposts_cnt',
       'comments_cnt', 'posts_count', 'likes_cnt_square', 'views_cnt_square',
       'reposts_cnt_square', 'comments_cnt_square', 'posts_count_square',
       'likes_cnt_poly_2', 'views_cnt_poly_2', 'reposts_cnt_poly_2',
       'comments_cnt_poly_2', 'posts_count_poly_2', 'likes_cnt_sqrt',
       'views_cnt_sqrt', 'reposts_cnt_sqrt', 'comments_cnt_sqrt',
       'posts_count_sqrt', 'log_likes_cnt', 'log_views_cnt', 'log_reposts_cnt',
       'log_comments_cnt', 'log_posts_count', 'recidivism_type_order',
       'guilty_plea_order', 'soft_bin', 'hard_bin', 'child_bin',
       'guilty_plea_0', 'guilty_plea_1', 'guilty_plea_2', 'recidivism_type_0',
       'recidivism_type_1', 'recidivism_type_2', 'recid

In [1282]:
data_reg_media_final = data_reg_media.merge(data[['ID', 'surname']], on='ID').drop(
    columns = ['Unnamed: 0']
).copy()

In [1290]:
data_reg_media_final[[
       'ID', 'gender_judge', 'recidivism_type', 'guilty_plea', 'soft_binary', 'hard_binary',
       'rel_type', 'year', 'children', 'sentence_period_whole', 'region',
       'Наименование суда', 'likes_cnt', 'views_cnt', 'reposts_cnt',
       'comments_cnt', 'posts_count', 'recidivism_type_order',
       'guilty_plea_order', 'soft_bin', 'hard_bin', 'child_bin',
       'guilty_plea_0', 'guilty_plea_1', 'guilty_plea_2', 'recidivism_type_0',
       'recidivism_type_1', 'recidivism_type_2', 'recidivism_type_3',
       'recidivism_type_4', 'gender_judge_bin', 'posts_bin',
       'sentence_period_whole_null', 'surname'
]].to_excel('База_данных_ВКР_Хамрин_Роман.xlsx')